# Web Spider for AFL Data
#### Quick Start

In [51]:
# import libraries
import pandas as pd
import requests
from fake_useragent import UserAgent
from time import sleep
import random

from IPython.core.display_functions import display

from bs4 import BeautifulSoup
import re

import warnings
from tqdm import tqdm

In [52]:
# Supresses scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# display the full tables
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

debug = True
random.seed(666)

In [3]:
# Query and Insert the old database !
games = pd.read_csv(r'./data/games.csv')
players = pd.read_csv(r'./data/players.csv')
stats = pd.read_csv(r'./data/stats.csv')

In [4]:
if debug is True:
    display(games.head())

,gameId,year,round,date,venue,startTime,attendance,homeTeam,homeTeamScore,awayTeam,awayTeamScore,rainfall
0,2021R101,2021,R1,18-Mar-2021,M.C.G.,7:25 PM,49218,Richmond,105,Carlton,80,0.00
1,2021R102,2021,R1,19-Mar-2021,M.C.G.,7:50 PM,46051,Collingwood,53,Western Bulldogs,69,0.00
2,2021R103,2021,R1,20-Mar-2021,M.C.G.,1:45 PM,21365,Melbourne,80,Fremantle,58,0.00
3,2021R104,2021,R1,20-Mar-2021,Adelaide Oval,4:05 PM,26985,Adelaide,103,Geelong,91,0.00
4,2021R105,2021,R1,20-Mar-2021,Docklands,7:25 PM,25128,Essendon,91,Hawthorn,92,0.00


In [5]:
if debug is True:
    display(players.head())

,playerId,displayName,height,weight,dob,position,origin
0,2021661124,"Berry, Sam",181,81,12-Feb-2002,"Midfield, Forward",Gippsland Power
1,2012662083,"Brown, Luke",181,81,22-Sep-1992,Defender,Norwood
2,2020665315,"Butts, Jordon",198,83,31-Dec-1999,Defender,Murray Bushrangers
3,2021678665,"Cook, Brayden",189,80,18-Jul-2002,"Midfield, Forward",South Adelaide
4,2019683521,"Davis, Ben",187,85,19-May-1997,Forward,UNSW


In [6]:
if debug is True:
    display(stats.head())

,gameId,team,year,round,playerId,displayName,gameNumber,Disposals,Kicks,Marks,Handballs,Goals,Behinds,Hit Outs,Tackles,Rebounds,Inside 50s,Clearances,Clangers,Frees,Frees Against,Brownlow Votes,Contested Possessions,Uncontested Possessions,Contested Marks,Marks Inside 50,One Percenters,Bounces,Goal Assists,% Played,Subs
0,2021R104,Adelaide,2021,R1,2021661124,"Berry, Sam",1,8,6,1,2,0,1,0,7,0,2,3,2,2,2,0,4,5,0,0,0,0,1,80,-
1,2021R104,Adelaide,2021,R1,2012662083,"Brown, Luke",168,5,2,0,3,0,0,0,2,0,0,0,0,1,0,0,2,3,0,0,0,0,0,23,Off
2,2021R104,Adelaide,2021,R1,2020665315,"Butts, Jordon",3,10,5,3,5,0,0,0,0,5,0,0,3,1,0,0,5,5,1,0,8,0,0,93,-
3,2021R104,Adelaide,2021,R1,2018689604,"Doedee, Tom",31,13,9,4,4,0,0,0,4,5,1,1,0,2,0,0,8,6,0,0,7,0,0,84,-
4,2021R104,Adelaide,2021,R1,2018703883,"Frampton, Billy",9,14,10,8,4,2,2,7,0,2,0,0,1,0,0,0,5,9,3,4,2,0,0,90,-


In [7]:
def query_player_info(url):

    # Crawling Website
    url_player= url
    # Fake a user header
    headers = {'User-Agent': UserAgent().random}

    player_datas = requests.get(url=url_player, headers=headers)
    html_players = player_datas.text
    # For Loop crawling, time interval is demanded.
    sleep(0.25)
    soup_players = BeautifulSoup(html_players, 'lxml')
    c_play_infos = soup_players.find_all('center')

    temp_player_info = re.match(r'.*?Born:(\d*-\w*-\d*) \(Debut.*?Height:(\d*).*?Weight:(\d*) kg', c_play_infos[0].text, flags=re.S)

    player_dob = temp_player_info.group(1)
    player_height = temp_player_info.group(2)
    player_weight = temp_player_info.group(3)

    return [player_dob, int(player_height), int(player_weight)]


In [8]:
def query_stats_info(new_stats_data, new_players_data, url, gameid, round, home_team, away_team):

    p_data = new_players_data
    s_data = new_stats_data

    # Crawling Website
    url= url
    gameid = gameid
    round = round

    # Fake a user header
    headers = {'User-Agent': UserAgent().random}

    # query stats data
    stats_datas = requests.get(url=url, headers=headers)
    html_stats = stats_datas.text

    # For Loop crawling, time interval is demanded.
    sleep(0.25)

    soup_stats = BeautifulSoup(html_stats, 'lxml')
    c_stats_tables = soup_stats.find_all('table', attrs={'class' : "sortable"})

    for i_table in range(0,2):
        # each table start here
        if i_table == 0:
            player_team = home_team
        if i_table == 1:
            player_team = away_team

        table_body = c_stats_tables[i_table].find_all('tbody')
        table_rows = table_body[0].find_all('tr')

        for table_row in table_rows:
            # each row start here

            table_columns = table_row.find_all('td')
            for p in range(len(table_columns)):
                # each column start here
                infos = table_columns[p].text
                if p != 0 and p != 1:
                    if infos == '\xa0':
                        infos = 0

                    infos = int(infos)

                if p == 0:
                    infos =  re.sub(r'\D', '', infos)
                    player_number = int(infos)
                    subs = '-'

                    table_players_rows = c_stats_tables[i_table+2].find_all('tbody')[0].find_all('tr')
                    for table_players_row in table_players_rows:
                        table_players_columns = table_players_row.find_all('td')
                        if int(table_players_columns[p].text) == player_number:
                            gameNumber = re.match(r'(\d*) \(', table_players_columns[3].text)[1]
                            gameNumber = int(gameNumber)

                if p == 1:
                    displayName = infos

                    name_url = 'https://afltables.com/afl/stats' + table_columns[p].find_all('a', href=True)[0]['href'][5:]
                    team_player_info = query_player_info(name_url)
                    player_dob = team_player_info[0]
                    player_height = team_player_info[1]
                    player_weight = team_player_info[2]

                    selected_p = p_data[(p_data['displayName'] == displayName) & (p_data['dob'] == player_dob)]

                    if selected_p.shape[0] == 0:
                        selected_p.at[0, 'playerId'] = int('2022' + str(random.randint(0,999999)).zfill(6))
                        selected_p.at[0, 'displayName'] = displayName
                        selected_p.at[0, 'height'] = player_height
                        selected_p.at[0, 'weight'] = player_weight
                        selected_p.at[0, 'dob'] = player_dob
                        selected_p.at[0, 'position'] = 'To_be_insert_manually'
                        selected_p.at[0, 'origin'] = 'To_be_insert_manually'
                        p_data = pd.concat([selected_p, p_data])

                    playerId = selected_p['playerId'].iloc[0]

                if p == 2:
                    Kicks = infos
                if p == 3:
                    Marks = infos
                if p == 4:
                    Handballs = infos
                if p == 5:
                    Disposals = infos
                if p == 6:
                    Goals = infos
                if p == 7:
                    Behinds = infos
                if p == 8:
                    Hit_Outs = infos
                if p == 9:
                    Tackles = infos
                if p == 10:
                    Rebounds = infos
                if p == 11:
                    Inside_50s = infos
                if p == 12:
                    Clearances = infos
                if p == 13:
                    Clangers = infos
                if p == 14:
                    Frees = infos
                if p == 15:
                    Frees_Against = infos
                if p == 16:
                    Brownlow_Votes = infos
                if p == 17:
                    Contested_Possessions = infos
                if p == 18:
                    Uncontested_Possessions = infos
                if p == 19:
                    Contested_Marks = infos
                if p == 20:
                    Marks_Inside_50 = infos
                if p == 21:
                    One_Percenters = infos
                if p == 22:
                    Bounces = infos
                if p == 23:
                    Goal_Assists = infos
                if p == 24:
                    p_Played = infos

            temp_stats_row = pd.DataFrame()
            temp_stats_row.at[0, 'gameId'] = gameid
            temp_stats_row.at[0, 'team'] = player_team
            temp_stats_row.at[0, 'year'] = 2022
            temp_stats_row.at[0, 'round'] = round
            temp_stats_row.at[0, 'playerId'] = playerId
            temp_stats_row.at[0, 'displayName'] = displayName
            temp_stats_row.at[0, 'gameNumber'] = gameNumber
            temp_stats_row.at[0, 'Disposals'] = Disposals
            temp_stats_row.at[0, 'Kicks'] = Kicks
            temp_stats_row.at[0, 'Marks'] = Marks
            temp_stats_row.at[0, 'Handballs'] = Handballs
            temp_stats_row.at[0, 'Goals'] = Goals
            temp_stats_row.at[0, 'Behinds'] = Behinds
            temp_stats_row.at[0, 'Hit Outs'] = Hit_Outs
            temp_stats_row.at[0, 'Tackles'] = Tackles
            temp_stats_row.at[0, 'Rebounds'] = Rebounds
            temp_stats_row.at[0, 'Inside 50s'] = Inside_50s
            temp_stats_row.at[0, 'Clearances'] = Clearances
            temp_stats_row.at[0, 'Clangers'] = Clangers
            temp_stats_row.at[0, 'Frees'] = Frees
            temp_stats_row.at[0, 'Frees Against'] = Frees_Against
            temp_stats_row.at[0, 'Brownlow Votes'] = Brownlow_Votes
            temp_stats_row.at[0, 'Contested Possessions'] = Contested_Possessions
            temp_stats_row.at[0, 'Uncontested Possessions'] = Uncontested_Possessions
            temp_stats_row.at[0, 'Contested Marks'] = Contested_Marks
            temp_stats_row.at[0, 'Marks Inside 50'] = Marks_Inside_50
            temp_stats_row.at[0, 'One Percenters'] = One_Percenters
            temp_stats_row.at[0, 'Bounces'] = Bounces
            temp_stats_row.at[0, 'Goal Assists'] = Goal_Assists
            temp_stats_row.at[0, '% Played'] = p_Played
            temp_stats_row.at[0, 'Subs'] = '-'
            s_data = pd.concat([temp_stats_row, s_data])

    return s_data, p_data


In [9]:
def query_games_info(new_stats_data, new_players_data, new_games_data, crawl_range_start_round, crawl_range_end_round):

    # compile Re
    re_date_time_venue = re.compile(r'\w* (\d*-\w*-\d*) (\d*:\d*\s\w*) .*?Venue: (.*)')
    re_att = re.compile(r'Att: \d*,*\d*')
    re_find_player_info = re.compile(r'.*?Born:(\d*-\w*-\d*) \(Debut.*?Height:(\d*).*?Weight:(\d*) kg')

    # Crawling Website
    url='https://afltables.com/afl/seas/2022.html'
    # Fake a user header
    headers = {'User-Agent': UserAgent().random}

    # Request the data
    datas = requests.get(url=url, headers=headers)
    html = datas.text

    # For Loop crawling, time interval is demanded.
    sleep(1)

    soup = BeautifulSoup(html, 'lxml')
    c_games = soup.find_all('table', attrs={'style':'font: 12px Verdana;', 'width':"100%", 'border':"1"})

    # each i is a table
    for i in tqdm(range(crawl_range_start_round, crawl_range_end_round * 10)):

        if i % 10 != 9:

            tables = c_games[i].find_all('td')

            r = i // 10 + 1
            id = i % 10 + 1

            gameid = '2022R' + str(r) + '0' + str(id)
            year = 2022
            round = 'R' + str(r)

            for j in range(len(tables)):

                if j == 0:
                    home_team = tables[j].text
                    home_team_index = 'https://afltables.com/afl' + tables[j].find_all('a', href=True)[0]['href'][2:]

                if j == 2:
                    home_team_score = int(tables[j].text)

                if j == 3:
                    temp = re_date_time_venue.match(tables[j].text)
                    date = temp.group(1)
                    venue = temp.group(3)
                    start_time = temp.group(2)

                    attendance = re_att.findall(tables[j].text)
                    if not attendance:
                        attendance = ['0']
                    attendance = re.sub(r'\D', '', attendance[0])
                    attendance = int(attendance)

                if j == 4:
                    away_team = tables[j].text
                    away_team_index = 'https://afltables.com/afl' + tables[j].find_all('a', href=True)[0]['href'][2:]

                if j == 6:
                    away_team_score = int(tables[j].text)

                if j == 7:
                    stats_link = 'https://afltables.com/afl' + tables[j].find_all('a', href=True)[0]['href'][2:]
                    new_stats_data, new_players_data = query_stats_info(new_stats_data, new_players_data, stats_link, gameid, round, home_team, away_team)

            rainfall = 0

            temp_games_row = pd.DataFrame()
            temp_games_row.at[0, 'gameId'] = gameid
            temp_games_row.at[0, 'year'] = year
            temp_games_row.at[0, 'round'] = round
            temp_games_row.at[0, 'date'] = date
            temp_games_row.at[0, 'venue'] = venue
            temp_games_row.at[0, 'startTime'] = start_time
            temp_games_row.at[0, 'attendance'] = attendance
            temp_games_row.at[0, 'homeTeam'] = home_team
            temp_games_row.at[0, 'homeTeamScore'] = home_team_score
            temp_games_row.at[0, 'awayTeam'] = away_team
            temp_games_row.at[0, 'awayTeamScore'] = away_team_score
            temp_games_row.at[0, 'rainfall'] = rainfall
            new_games_data = pd.concat([temp_games_row, new_games_data])


    return new_stats_data, new_players_data, new_games_data


In [10]:
new_players_data = players.copy()
new_stats_data = stats.copy()
new_games_data= games.copy()

In [11]:
new_stats_data, new_players_data, new_games_data = query_games_info(new_stats_data, new_players_data, new_games_data, 0,3)

100%|██████████| 30/30 [12:17<00:00, 24.58s/it]


In [17]:
new_stats_data.to_csv('./output/stats.csv', index=False)
new_players_data.to_csv('./output/players.csv', index=False)
new_games_data.to_csv('./output/games.csv', index=False)

In [12]:
'''# m = re.match(r'\w* (\d*\-\w*\-\d*) (\d*\:\d*\s\w*) \w*\: (\d*\,\d*) \w*\: (.*)', 'Wed 16-Mar-2022 7:10 PM Att: 58,002 Venue: M.C.G.')

m = re.match(r'\w* (\d*-\w*-\d*) (\d*:\d*\s\w*) .*?Venue: (.*)', 'Wed 16-Mar-2022 7:10 PM Att: 58,002 Venue: M.C.G.')

'Sun 03-Apr-2022 4:20 PM (6:20 PM) Venue: Perth Stadium'

'Sun 20-Mar-2022 4:40 PM (6:40 PM) Att: 20,932 Venue: Perth Stadium'
# m = re_date_time_att_venue.match('Wed 16-Mar-2022 7:10 PM Att: 58,002 Venue: M.C.G.')


re_att = re.compile(r'Att: \d*,*\d*')

nn = re_att.findall('Sun 20-Mar-2022 4:40 PM (6:40 PM) Att: 20,932 Venue: Perth Stadium')
if not nn:
    nn = ['0']
nn = re.sub(r'\D', '', nn[0])
nn = int(nn)

# nn = re.findall(r'Att: \d*,*\d*', 'Sun 03-Apr-2022 4:20 PM (6:20 PM) Venue: Perth Stadium')


print(m)
print(m.group(1))
print(m.group(2))


# n = m.group(3)
# n = re.sub(r'\D', '', n)


# print(n)
print(m.group(3))

print(nn)'''

"# m = re.match(r'\\w* (\\d*\\-\\w*\\-\\d*) (\\d*\\:\\d*\\s\\w*) \\w*\\: (\\d*\\,\\d*) \\w*\\: (.*)', 'Wed 16-Mar-2022 7:10 PM Att: 58,002 Venue: M.C.G.')\n\nm = re.match(r'\\w* (\\d*-\\w*-\\d*) (\\d*:\\d*\\s\\w*) .*?Venue: (.*)', 'Wed 16-Mar-2022 7:10 PM Att: 58,002 Venue: M.C.G.')\n\n'Sun 03-Apr-2022 4:20 PM (6:20 PM) Venue: Perth Stadium'\n\n'Sun 20-Mar-2022 4:40 PM (6:40 PM) Att: 20,932 Venue: Perth Stadium'\n# m = re_date_time_att_venue.match('Wed 16-Mar-2022 7:10 PM Att: 58,002 Venue: M.C.G.')\n\n\nre_att = re.compile(r'Att: \\d*,*\\d*')\n\nnn = re_att.findall('Sun 20-Mar-2022 4:40 PM (6:40 PM) Att: 20,932 Venue: Perth Stadium')\nif not nn:\n    nn = ['0']\nnn = re.sub(r'\\D', '', nn[0])\nnn = int(nn)\n\n# nn = re.findall(r'Att: \\d*,*\\d*', 'Sun 03-Apr-2022 4:20 PM (6:20 PM) Venue: Perth Stadium')\n\n\nprint(m)\nprint(m.group(1))\nprint(m.group(2))\n\n\n# n = m.group(3)\n# n = re.sub(r'\\D', '', n)\n\n\n# print(n)\nprint(m.group(3))\n\nprint(nn)"